In [1]:
source = "CombineSignals";

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
params = read_params(source);

In [4]:
X = zeros(num_items(), length(params["alphas"]))
for j = 1:size(X)[2]
    X[:, j] = read_recommendee_alpha(params["alphas"][j]).rating
end

In [5]:
predictions = X * params["β"];

In [6]:
model(items) = predictions[items];

In [7]:
write_recommendee_alpha(model);